In [0]:
#Import Libraries
#import mlflow
#import mlflow.spark
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
#from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import PolynomialExpansion, PCA
from pyspark.sql.functions import length, col, explode
from pyspark.ml.functions import vector_to_array



##Basic EDA

In [0]:
#Read parquet 
data = spark.read.parquet("/tmp/output/higgsdf.parquet")

In [0]:
#Check data
data.show(5)

+-----+------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+--------------------+-------------------+-----------------+-------------------+--------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
label| feature1| feature2| feature3| feature4| feature5| feature6| feature7| feature8| feature9| feature10| feature11| feature12| feature13| feature14| feature15| feature16| feature17| feature18| feature19| feature20| feature21| feature22| feature23| feature24| feature25| feature26| feature27| feature28|
+-----+------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+--------------------+-------------------+-----------------+-------------------+--------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
 1.0| 0.869293212890625|-0.6350818276405334|0.22569026052951813| 0.327470064163208|-0.6899932026863098|0.7542022466659546|-0.24857313930988312|-1.0920639038085938| 0.0|1.3749921321868896|-0.6536741852760315| 0.9303491115570068|1.1074360609054565| 1.138904333114624| -1.5781983137130737| -1.046985387802124| 0.0| 0.657929539680481|-0.01045456994324...| -0.0457671694457531|3.101961374282837| 1.353760004043579|0.9795631170272827| 0.978076159954071|0.9200048446655273|0.7216574549674988|0.9887509346008301|0.8766783475875854|
 1.0|0.9075421094894409| 0.3291472792625427| 0.3594118654727936|1.4979698657989502|-0.3130095303058624| 1.09553062915802| -0.5575249195098877| -1.588229775428772|2.1730761528015137|0.8125811815261841|-0.2136419266462326| 1.2710145711898804| 2.214872121810913|0.4999939501285553| -1.2614318132400513| 0.7321561574935913| 0.0|0.39870089292526245| -1.138930082321167|-8.19110195152461...| 0.0|0.3022198975086212|0.8330481648445129|0.9856996536254883|0.9780983924865723|0.7797321677207947|0.9923557639122009|0.7983425855636597|
 1.0|0.7988347411155701| 1.4706387519836426|-1.6359747648239136| 0.45377317070961| 0.4256291687488556| 1.104874610900879| 1.2823222875595093| 1.3816642761230469| 0.0|0.8517372012138367| 1.540658950805664|-0.8196895122528076| 2.214872121810913|0.9934899210929871| 0.3560801148414612|-0.2087775468826294|2.548224449157715| 1.256954550743103| 1.128847599029541| 0.9004608392715454| 0.0|0.9097532629966736| 1.108330488204956|0.9856922030448914|0.9513312578201294|0.8032515048980713|0.8659244179725647|0.7801175713539124|
 0.0| 1.344384789466858|-0.8766260147094727| 0.9359127283096313| 1.992050051689148| 0.8824543952941895| 1.786065936088562| -1.6467777490615845|-0.9423825144767761| 0.0| 2.423264741897583|-0.6760157942771912| 0.7361586689949036| 2.214872121810913|1.2987197637557983| -1.430738091468811|-0.3646581768989563| 0.0| 0.7453126907348633| -0.6783788204193115| -1.360356330871582| 0.0|0.9466524720191956|1.0287036895751953|0.9986560940742493|0.7282806038856506|0.8692002296447754|1.0267364978790283|0.9579039812088013|
 1.0|1.1050089597702026|0.32135555148124695| 1.522401213645935|0.8828076124191284| -1.205349326133728|0.6814661026000977| -1.0704638957977295|-0.9218706488609314| 0.0|0.8008721470832825| 1.020974040031433| 0.9714065194129944| 2.214872121810913|0.5967612862586975|-0.35027286410331726| 0.6311942934989929| 0.0| 0.4799988865852356|-0.37356552481651306| 0.11304061114788055| 0.0| 0.755856454372406|1.3610570430755615| 0.986609697341919|0.8380846381187439|1.1332951784133911|0.8722448945045471|0.8084865212440491|
+

In [0]:
#Check the label distribution
labelcounts = data.groupBy('label').count().show()

+-----+-------+
label| count|
+-----+-------+
 0.0|5170877|
 1.0|5829123|
+-----+-------+

In [0]:
# import sql function pyspark
import pyspark.sql.functions as f

# null values in each column
data_agg = data.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in data.columns])
data_agg.show()

+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
label|feature1|feature2|feature3|feature4|feature5|feature6|feature7|feature8|feature9|feature10|feature11|feature12|feature13|feature14|feature15|feature16|feature17|feature18|feature19|feature20|feature21|feature22|feature23|feature24|feature25|feature26|feature27|feature28|
+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+

In [0]:
features =['feature1','feature2','feature3','feature4','feature5','feature6','feature7','feature8','feature9','feature10','feature11','feature12','feature13','feature14','feature15','feature16','feature17','feature18','feature19','feature20','feature21','feature22','feature23','feature24','feature25','feature26','feature27','feature28']

##Vanilla Pipeline

In [0]:
stages = [] #stages in pipeline

In [0]:
#Start by using vector assembler to truncate all the columns
vecAssembler = VectorAssembler(inputCols = features, outputCol="features")
stages += [vecAssembler]

In [0]:
#Check to see vector assembler output

testdf = vecAssembler.transform(data)
testdf.head().features

Out[10]: DenseVector([0.8693, -0.6351, 0.2257, 0.3275, -0.69, 0.7542, -0.2486, -1.0921, 0.0, 1.375, -0.6537, 0.9303, 1.1074, 1.1389, -1.5782, -1.047, 0.0, 0.6579, -0.0105, -0.0458, 3.102, 1.3538, 0.9796, 0.9781, 0.92, 0.7217, 0.9888, 0.8767])

In [0]:
#Scale the data
scaler = MinMaxScaler( min=-1.0, max=1.0,inputCol="features", outputCol="scaledFeatures")
stages += [scaler]
stages

Out[11]: [VectorAssembler_38db8ca6adb0, MinMaxScaler_0a9081d859dd]

In [0]:
#Test the scaler
# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(testdf)

# rescale each feature to range [min, max].
testdf2 = scalerModel.transform(testdf)

In [0]:
#Build baseline classifier
lr = LogisticRegression(labelCol="label",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.3)

stages += [lr]

In [0]:
#Split train and test label
train,test = data.randomSplit([0.8, 0.2])

In [0]:
#Check the ratio, if balanced no need to do stratified sampling
train.groupBy('label').count().show()

+-----+-------+
label| count|
+-----+-------+
 0.0|4136941|
 1.0|4664734|
+-----+-------+

In [0]:


#Build the pipeline
#with mlflow.start_run(run_name='logreg_baseline') as run:
Regpipeline = Pipeline(stages = stages) 
  
model = Regpipeline.fit(train)
  
predicted = model.transform(test)
  
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC', numBins=1000)

accuracy = evaluator.evaluate(predicted)



In [0]:
#Record model training times in dictionary for comparative analysis later
model_metrics = { }
model_metrics['Baseline'] = {}
model_metrics['Baseline']['Time'] = 5.80
model_metrics['Baseline']['Features'] = 28
model_metrics['Baseline']['AUC'] = 0.66


print(model_metrics)

{'Baseline': {'Time': 5.8, 'Features': 28, 'AUC': 0.66}}

In [0]:
predicted = model.transform(train)

In [0]:
predicted.select('features', 'label', 'rawPrediction', 'probability', 'prediction').show()

+--------------------+-----+--------------------+--------------------+----------+
 features|label| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[0.27469664812088...| 0.0|[0.00843005004345...|[0.50210750002995...| 0.0|
[0.27469664812088...| 0.0|[-0.1656073204684...|[0.45869253422925...| 1.0|
[0.27469664812088...| 0.0|[-0.2878721826824...|[0.42852487159626...| 1.0|
[0.27469664812088...| 0.0|[-0.0964503119186...|[0.47590609724584...| 1.0|
[0.27469664812088...| 0.0|[-0.0085673520800...|[0.49785817508070...| 1.0|
[0.27469664812088...| 0.0|[0.08564011149672...|[0.52139695196086...| 0.0|
[0.27469664812088...| 0.0|[0.43593343831387...|[0.60728962506287...| 0.0|
[0.27469664812088...| 0.0|[-0.0101214056505...|[0.49746967018850...| 1.0|
[0.27469664812088...| 0.0|[0.69056074772823...|[0.66609165626476...| 0.0|
[0.27469664812088...| 0.0|[-0.1123163375238...|[0.47195039647271...| 1.0|
[0.27469664812088...| 0.0|[-0.2359698610026...|[0.44127975285720...| 1.0|
[0.27469664812088...| 0.0|[-0.1421038994497...|[0.46453368752953...| 1.0|
[0.27469664812088...| 0.0|[0.09706939528682...|[0.52424831190018...| 0.0|
[0.27469664812088...| 0.0|[0.16647912747316...|[0.54152392214299...| 0.0|
[0.27469664812088...| 0.0|[0.02653745974610...|[0.50663397561696...| 0.0|
[0.27469664812088...| 0.0|[0.28872546499154...|[0.57168407706667...| 0.0|
[0.27469664812088...| 0.0|[-0.1527891444987...|[0.46187684893304...| 1.0|
[0.27469664812088...| 0.0|[0.20188310832158...|[0.55030005395649...| 0.0|
[0.27469664812088...| 0.0|[-0.0615435902059...|[0.48461895692925...| 1.0|
[0.27469664812088...| 0.0|[-0.1922690457407...|[0.45208026995424...| 1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:


evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC', numBins=1000)

accuracy = evaluator.evaluate(predicted)

In [0]:
print(accuracy)

0.6559305511237383

# Pipeline 2nd Iteration - observe classifier change

In [0]:
stages2 =[]
stages2 += [vecAssembler]

In [0]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
stages2 += [rf]

In [0]:
Rfpipeline = Pipeline(stages = stages2) 
  
model = Rfpipeline.fit(train)
  
predicted = model.transform(test)

In [0]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC', numBins=1000)

accuracy = evaluator.evaluate(predicted)

print(accuracy)

0.733801175529573

In [0]:
model_metrics['RFBaseline'] = {}
model_metrics['RFBaseline']['Time'] = 9.95
model_metrics['RFBaseline']['Features'] = 28
model_metrics['RFBaseline']['AUC'] = 0.73
print(model_metrics)

{'Baseline': {'Time': 5.8, 'Features': 28, 'AUC': 0.66}, 'RFBaseline': {'Time': 9.95, 'Features': 28, 'AUC': 0.73}}

#Pipeline 3rd Iteration - Feature Expansion

In [0]:
#Initialise stages of 3rd pipeline
stages3 = []
stages3 += [vecAssembler]

In [0]:
#Instantiate polyexpansion object
polyExpansion = PolynomialExpansion(degree=2, inputCol="features", outputCol="polyFeatures")

#Add object to stages
stages3 += [polyExpansion]

In [0]:
rf2 = RandomForestClassifier(featuresCol = 'polyFeatures', labelCol = 'label')

In [0]:
#Carryover randomforestclassifier object from previous pipeline over
stages3 += [rf2]

#Check stages
stages3

Out[14]: [VectorAssembler_368d3ddc5dae,
 PolynomialExpansion_30021f843e89,
 RandomForestClassifier_cf95764e0a68]

In [0]:
#Build pipeline 3
pipeline3 = Pipeline(stages = stages3) 
  
model3 = pipeline3.fit(train)
  
predicted3 = model3.transform(test)

In [0]:
#Check the number of features that was blown out by the 2nd degree poly expansion
predicted3.head()

Out[21]: Row(label=0.0, feature1=0.2746966481208801, feature2=-2.032726764678955, feature3=-1.6903512477874756, feature4=0.9556527733802795, feature5=0.19095838069915771, feature6=0.8881319165229797, feature7=-0.2277783304452896, feature8=-1.5072909593582153, feature9=2.1730761528015137, feature10=0.8751552104949951, feature11=0.9850332736968994, feature12=0.4737241566181183, feature13=2.214872121810913, feature14=0.7291797995567322, feature15=0.09665925055742264, feature16=-0.9304909706115723, feature17=0.0, feature18=0.9495312571525574, feature19=-1.9517656564712524, feature20=0.5791667699813843, feature21=0.0, feature22=0.8406495451927185, feature23=0.9791290163993835, feature24=0.990804135799408, feature25=1.032375454902649, feature26=1.5314446687698364, feature27=0.9710801839828491, feature28=0.8907249569892883, features=DenseVector([0.2747, -2.0327, -1.6904, 0.9557, 0.191, 0.8881, -0.2278, -1.5073, 2.1731, 0.8752, 0.985, 0.4737, 2.2149, 0.7292, 0.0967, -0.9305, 0.0, 0.9495, -1.9518, 0.5792, 0.0, 0.8406, 0.9791, 0.9908, 1.0324, 1.5314, 0.9711, 0.8907]), polyFeatures=DenseVector([0.2747, 0.0755, -2.0327, -0.5584, 4.132, -1.6904, -0.4643, 3.436, 2.8573, 0.9557, 0.2625, -1.9426, -1.6154, 0.9133, 0.191, 0.0525, -0.3882, -0.3228, 0.1825, 0.0365, 0.8881, 0.244, -1.8053, -1.5013, 0.8487, 0.1696, 0.7888, -0.2278, -0.0626, 0.463, 0.385, -0.2177, -0.0435, -0.2023, 0.0519, -1.5073, -0.414, 3.0639, 2.5479, -1.4404, -0.2878, -1.3387, 0.3433, 2.2719, 2.1731, 0.5969, -4.4173, -3.6733, 2.0767, 0.415, 1.93, -0.495, -3.2755, 4.7223, 0.8752, 0.2404, -1.779, -1.4793, 0.8363, 0.1671, 0.7773, -0.1993, -1.3191, 1.9018, 0.7659, 0.985, 0.2706, -2.0023, -1.6651, 0.9413, 0.1881, 0.8748, -0.2244, -1.4847, 2.1406, 0.8621, 0.9703, 0.4737, 0.1301, -0.963, -0.8008, 0.4527, 0.0905, 0.4207, -0.1079, -0.714, 1.0294, 0.4146, 0.4666, 0.2244, 2.2149, 0.6084, -4.5022, -3.7439, 2.1166, 0.4229, 1.9671, -0.5045, -3.3385, 4.8131, 1.9384, 2.1817, 1.0492, 4.9057, 0.7292, 0.2003, -1.4822, -1.2326, 0.6968, 0.1392, 0.6476, -0.1661, -1.0991, 1.5846, 0.6381, 0.7183, 0.3454, 1.615, 0.5317, 0.0967, 0.0266, -0.1965, -0.1634, 0.0924, 0.0185, 0.0858, -0.022, -0.1457, 0.21, 0.0846, 0.0952, 0.0458, 0.2141, 0.0705, 0.0093, -0.9305, -0.2556, 1.8914, 1.5729, -0.8892, -0.1777, -0.8264, 0.2119, 1.4025, -2.022, -0.8143, -0.9166, -0.4408, -2.0609, -0.6785, -0.0899, 0.8658, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9495, 0.2608, -1.9301, -1.605, 0.9074, 0.1813, 0.8433, -0.2163, -1.4312, 2.0634, 0.831, 0.9353, 0.4498, 2.1031, 0.6924, 0.0918, -0.8835, 0.0, 0.9016, -1.9518, -0.5361, 3.9674, 3.2992, -1.8652, -0.3727, -1.7334, 0.4446, 2.9419, -4.2413, -1.7081, -1.9226, -0.9246, -4.3229, -1.4232, -0.1887, 1.8161, 0.0, -1.8533, 3.8094, 0.5792, 0.1591, -1.1773, -0.979, 0.5535, 0.1106, 0.5144, -0.1319, -0.873, 1.2586, 0.5069, 0.5705, 0.2744, 1.2828, 0.4223, 0.056, -0.5389, 0.0, 0.5499, -1.1304, 0.3354, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8406, 0.2309, -1.7088, -1.421, 0.8034, 0.1605, 0.7466, -0.1915, -1.2671, 1.8268, 0.7357, 0.8281, 0.3982, 1.8619, 0.613, 0.0813, -0.7822, 0.0, 0.7982, -1.6408, 0.4869, 0.0, 0.7067, 0.9791, 0.269, -1.9903, -1.6551, 0.9357, 0.187, 0.8696, -0.223, -1.4758, 2.1277, 0.8569, 0.9645, 0.4638, 2.1686, 0.714, 0.0946, -0.9111, 0.0, 0.9297, -1.911, 0.5671, 0.0, 0.8231, 0.9587, 0.9908, 0.2722, -2.014, -1.6748, 0.9469, 0.1892, 0.88, -0.2257, -1.4934, 2.1531, 0.8671, 0.976, 0.4694, 2.1945, 0.7225, 0.0958, -0.9219, 0.0, 0.9408, -1.9338, 0.5738, 0.0, 0.8329, 0.9701, 0.9817, 1.0324, 0.2836, -2.0985, -1.7451, 0.9866, 0.1971, 0.9169, -0.2352, -1.5561, 2.2434, 0.9035, 1.0169, 0.4891, 2.2866, 0.7528, 0.0998, -0.9606, 0.0, 0.9803, -2.015, 0.5979, 0.0, 0.8679, 1.0108, 1.0229, 1.0658, 1.5314, 0.4207, -3.113, -2.5887, 1.4635, 0.2924, 1.3601, -0.3488, -2.3083, 3.3279, 1.3403, 1.5085, 0.7255, 3.392, 1.1167, 0.148, -1.425, 0.0, 1.4542, -2.989, 0.887, 0.0, 1.2874, 1.4

In [0]:
evaluator3 = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC', numBins=1000)

accuracy = evaluator3.evaluate(predicted3)

In [0]:
print(accuracy)

0.7399004902836516

In [0]:
model_metrics['RFPoly1'] = {}
model_metrics['RFPoly1']['Time'] = 34
model_metrics['RFPoly1']['Features'] = 414
model_metrics['RFPoly1']['AUC'] = 0.74
print(model_metrics)

{'Baseline': {'Time': 5.8, 'Features': 28, 'AUC': 0.66}, 'RFBaseline': {'Time': 9.95, 'Features': 28, 'AUC': 0.73}, 'RFPoly1': {'Time': 34, 'Features': 414, 'AUC': 0.74}}

#Pipeline Iteration 4(4th degree polynomial expansion) (insufficient memory with community edition cluster will run on paid)

In [0]:
#Initialise stages of 4th pipeline
stages4 = []
stages4 += [vecAssembler]

In [0]:
#Instantiate polyexpansion object
polyExpansion2 = PolynomialExpansion(degree=3, inputCol="features", outputCol="polyFeatures")

#Add object to stages
stages4 += [polyExpansion2]

In [0]:

#Carryover randomforestclassifier object from previous pipeline over
stages4 += [rf2]

#Check stages
stages4

Out[22]: [VectorAssembler_38db8ca6adb0,
 PolynomialExpansion_1f4e8a7d9ec2,
 RandomForestClassifier_4ffbda3d8670]

In [0]:
#Build pipeline 4
pipeline4 = Pipeline(stages = stages4) 
  
model4 = pipeline4.fit(train)
  
predicted4 = model4.transform(test)

In [0]:
evaluator4 = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC', numBins=1000)
 
accuracy = evaluator4.evaluate(predicted4)

#Pipeline iteration 5 PCA on 2nd degree expansion

In [0]:
#Initialise stages of 4th pipeline
stages5 = []
stages5 += [vecAssembler]

In [0]:
#Instantiate polyexpansion object
polyExpansion3 = PolynomialExpansion(degree=2, inputCol="features", outputCol="polyFeatures")

#Add object to stages
stages5 += [polyExpansion3]

In [0]:
pca = PCA(k=5, inputCol="polyFeatures",outputCol="PCAfeatures")

stages5 += [pca]

In [0]:
rf3 = RandomForestClassifier(featuresCol = 'PCAfeatures', labelCol = 'label')

In [0]:
#Check stages before building pipelien
stages5 += [rf3]
stages5

Out[24]: [VectorAssembler_cf88a3964eaf,
 PolynomialExpansion_7be48b8beb2f,
 PCA_61e610e96775,
 RandomForestClassifier_14bae7c06fd3]

In [0]:
#Build pipeline 5
pipeline5 = Pipeline(stages = stages5) 
  
model5 = pipeline5.fit(train)
  
predicted5 = model5.transform(test)

In [0]:
evaluator5 = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC', numBins=1000)
 
accuracy = evaluator5.evaluate(predicted5)

In [0]:
predicted5.head()

Out[29]: Row(label=0.0, feature1=0.2746966481208801, feature2=-2.2353124618530273, feature3=0.9719787836074829, feature4=1.2100285291671753, feature5=0.5871968865394592, feature6=0.6360288858413696, feature7=0.3683401048183441, feature8=-0.9689925909042358, feature9=0.0, feature10=0.6515505909919739, feature11=-0.05045115947723389, feature12=0.7061978578567505, feature13=1.1074360609054565, feature14=1.0935940742492676, feature15=-0.7908332347869873, feature16=-1.032562255859375, feature17=2.548224449157715, feature18=0.38068878650665283, feature19=0.019527068361639977, feature20=1.307766318321228, feature21=0.0, feature22=0.8629414439201355, feature23=0.8080047965049744, feature24=0.9740193486213684, feature25=0.8999179601669312, feature26=0.9728993773460388, feature27=0.6810853481292725, feature28=0.6895888447761536, features=DenseVector([0.2747, -2.2353, 0.972, 1.21, 0.5872, 0.636, 0.3683, -0.969, 0.0, 0.6516, -0.0505, 0.7062, 1.1074, 1.0936, -0.7908, -1.0326, 2.5482, 0.3807, 0.0195, 1.3078, 0.0, 0.8629, 0.808, 0.974, 0.8999, 0.9729, 0.6811, 0.6896]), polyFeatures=DenseVector([0.2747, 0.0755, -2.2353, -0.614, 4.9966, 0.972, 0.267, -2.1727, 0.9447, 1.21, 0.3324, -2.7048, 1.1761, 1.4642, 0.5872, 0.1613, -1.3126, 0.5707, 0.7105, 0.3448, 0.636, 0.1747, -1.4217, 0.6182, 0.7696, 0.3735, 0.4045, 0.3683, 0.1012, -0.8234, 0.358, 0.4457, 0.2163, 0.2343, 0.1357, -0.969, -0.2662, 2.166, -0.9418, -1.1725, -0.569, -0.6163, -0.3569, 0.9389, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6516, 0.179, -1.4564, 0.6333, 0.7884, 0.3826, 0.4144, 0.24, -0.6313, 0.0, 0.4245, -0.0505, -0.0139, 0.1128, -0.049, -0.061, -0.0296, -0.0321, -0.0186, 0.0489, 0.0, -0.0329, 0.0025, 0.7062, 0.194, -1.5786, 0.6864, 0.8545, 0.4147, 0.4492, 0.2601, -0.6843, 0.0, 0.4601, -0.0356, 0.4987, 1.1074, 0.3042, -2.4755, 1.0764, 1.34, 0.6503, 0.7044, 0.4079, -1.0731, 0.0, 0.7216, -0.0559, 0.7821, 1.2264, 1.0936, 0.3004, -2.4445, 1.063, 1.3233, 0.6422, 0.6956, 0.4028, -1.0597, 0.0, 0.7125, -0.0552, 0.7723, 1.2111, 1.1959, -0.7908, -0.2172, 1.7678, -0.7687, -0.9569, -0.4644, -0.503, -0.2913, 0.7663, 0.0, -0.5153, 0.0399, -0.5585, -0.8758, -0.8649, 0.6254, -1.0326, -0.2836, 2.3081, -1.0036, -1.2494, -0.6063, -0.6567, -0.3803, 1.0005, 0.0, -0.6728, 0.0521, -0.7292, -1.1435, -1.1292, 0.8166, 1.0662, 2.5482, 0.7, -5.6961, 2.4768, 3.0834, 1.4963, 1.6207, 0.9386, -2.4692, 0.0, 1.6603, -0.1286, 1.7996, 2.822, 2.7867, -2.0152, -2.6312, 6.4934, 0.3807, 0.1046, -0.851, 0.37, 0.4606, 0.2235, 0.2421, 0.1402, -0.3689, 0.0, 0.248, -0.0192, 0.2688, 0.4216, 0.4163, -0.3011, -0.3931, 0.9701, 0.1449, 0.0195, 0.0054, -0.0436, 0.019, 0.0236, 0.0115, 0.0124, 0.0072, -0.0189, 0.0, 0.0127, -0.001, 0.0138, 0.0216, 0.0214, -0.0154, -0.0202, 0.0498, 0.0074, 0.0004, 1.3078, 0.3592, -2.9233, 1.2711, 1.5824, 0.7679, 0.8318, 0.4817, -1.2672, 0.0, 0.8521, -0.066, 0.9235, 1.4483, 1.4302, -1.0342, -1.3504, 3.3325, 0.4979, 0.0255, 1.7103, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8629, 0.237, -1.9289, 0.8388, 1.0442, 0.5067, 0.5489, 0.3179, -0.8362, 0.0, 0.5623, -0.0435, 0.6094, 0.9557, 0.9437, -0.6824, -0.891, 2.199, 0.3285, 0.0169, 1.1285, 0.0, 0.7447, 0.808, 0.222, -1.8061, 0.7854, 0.9777, 0.4745, 0.5139, 0.2976, -0.783, 0.0, 0.5265, -0.0408, 0.5706, 0.8948, 0.8836, -0.639, -0.8343, 2.059, 0.3076, 0.0158, 1.0567, 0.0, 0.6973, 0.6529, 0.974, 0.2676, -2.1772, 0.9467, 1.1786, 0.5719, 0.6195, 0.3588, -0.9438, 0.0, 0.6346, -0.0491, 0.6879, 1.0787, 1.0652, -0.7703, -1.0057, 2.482, 0.3708, 0.019, 1.2738, 0.0, 0.8405, 0.787, 0.9487, 0.8999, 0.2472, -2.0116, 0.8747, 1.0889, 0.5284, 0.5724, 0.3315, -0.872, 0.0, 0.5863, -0.0454, 0.6355, 0.9966, 0.9841, -0.7117, -0.9292, 2.2932, 0.3426, 0.0176, 1.1769, 0.0, 0.7766, 0.7271, 0.8765, 0.8099, 0.9729, 0.2673, -2.1747, 0.9456, 1.1772, 0.5713, 0.6188, 0.3584, -0.9427, 0.0, 0.6339, -0.0491, 0.6871, 1.0774, 1.064, -0.7694, -1.0046, 2.4792, 0.3704, 0.019, 1.2723, 0.0, 0.8396, 0.7861, 0.9476, 0.8755, 

In [0]:
print(accuracy)

0.5860876153990631

In [0]:
model_metrics['RFPolyPCA5'] = {}
model_metrics['RFPolyPCA5']['Time'] = 37
model_metrics['RFPolyPCA5']['Features'] = 5
model_metrics['RFPolyPCA5']['AUC'] = 0.59
print(model_metrics)

{'Baseline': {'Time': 5.8, 'Features': 28, 'AUC': 0.66}, 'RFBaseline': {'Time': 9.95, 'Features': 28, 'AUC': 0.73}, 'RFPoly1': {'Time': 34, 'Features': 414, 'AUC': 0.74}, 'RFPolyPCA5': {'Time': 37, 'Features': 5, 'AUC': 0.59}}

#PCA on original dataset

In [0]:
#Initialise stages of 6th pipeline
stages6 = []
stages6 += [vecAssembler]

In [0]:
pca = PCA(k=15, inputCol="features" ,outputCol="PCAfeatures")

stages6 += [pca]

In [0]:
stages6

Out[35]: [VectorAssembler_cf88a3964eaf, PCA_f3630d71c115]

In [0]:
rf4 = RandomForestClassifier(featuresCol = 'PCAfeatures', labelCol = 'label')
stages6 += [rf4]

In [0]:
#Build pipeline 6
pipeline6 = Pipeline(stages = stages6) 
  
model6 = pipeline6.fit(train)
  
predicted6 = model6.transform(test)

In [0]:
evaluator6 = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC', numBins=1000)
 
accuracy = evaluator6.evaluate(predicted6)

print(accuracy)

0.6104113042820289

In [0]:
model_metrics['RFPPCA15'] = {}
model_metrics['RFPPCA15']['Time'] = 14
model_metrics['RFPPCA15']['Features'] = 15
model_metrics['RFPPCA15']['AUC'] = 0.61
print(model_metrics)

{'Baseline': {'Time': 5.8, 'Features': 28, 'AUC': 0.66}, 'RFBaseline': {'Time': 9.95, 'Features': 28, 'AUC': 0.73}, 'RFPoly1': {'Time': 34, 'Features': 414, 'AUC': 0.74}, 'RFPolyPCA5': {'Time': 37, 'Features': 5, 'AUC': 0.59}, 'RFPPCA15': {'Time': 14, 'Features': 15, 'AUC': 0.61}}